Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

*Наименование вакансии
*Предлагаемую зарплату (отдельно мин. отдельно макс. и отдельно валюту)
*Ссылку на саму вакансию

In [1]:
import requests
import json
from pprint import pprint
from bs4 import BeautifulSoup as bs
import re
import unicodedata
import pandas as pd

In [2]:
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'}

In [3]:
vacancy_title = input('Введите наименование вакансии: ')

Введите наименование вакансии: аналитик


In [4]:
page = int(input('Введите номер страницы от 1 до 40 (если хотите посмотреть все страницы, введите 0): '))

Введите номер страницы от 1 до 40 (если хотите посмотреть все страницы, введите 0): 0


In [5]:
main_link_hh = (f'https://hh.ru/search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&text=')

In [6]:
vacancies = []

In [7]:
if 1 <= page <= 40:
    hh_page = page-1
    hh_response = requests.get(f'{main_link_hh}{vacancy_title}&page={hh_page}', headers = header)
    hh_html = hh_response.text
    hh_soup = bs(hh_html, 'lxml')
    hh_vacancy_block = hh_soup.find_all('div',{'class':"vacancy-serp"})[0]
    hh_vacancies_list = hh_vacancy_block.find_all('div',{'class':"vacancy-serp-item"})
    
    if hh_response.ok:
        
        for vacancy in hh_vacancies_list:
            hh_vacancy_data={}
            hh_vacancy_link = vacancy.find('a')['href']
            hh_vacancy_name = vacancy.find('span',{'class': 'resume-search-item__name'}).getText()
            hh_vacancy_salary = vacancy.find('div',{'class': 'vacancy-serp-item__sidebar'}).getText()
            hh_vacancy_salary = unicodedata.normalize("NFKD", hh_vacancy_salary)

            if 'от' in hh_vacancy_salary or '-' in hh_vacancy_salary:
                vacancy_min_salary = hh_vacancy_salary
                vacancy_min_salary = vacancy_min_salary[:vacancy_min_salary.find('-')]
            else:
                vacancy_min_salary = 'Nan'

            if 'до' in hh_vacancy_salary or '-' in hh_vacancy_salary:
                vacancy_max_salary = hh_vacancy_salary
                vacancy_max_salary = vacancy_max_salary[vacancy_max_salary.find('-')+1:]
            else:
                vacancy_max_salary = 'Nan'

            if ' руб' in hh_vacancy_salary or 'US' in hh_vacancy_salary or 'EU' in hh_vacancy_salary:
                vacancy_salary_currency = hh_vacancy_salary
                vacancy_salary_currency = vacancy_salary_currency[-4:]
                vacancy_salary_currency = vacancy_salary_currency.replace('.', '')
            else:
                vacancy_salary_currency = 'Nan'  
            
            if 'По договорённости' in hh_vacancy_salary:
                vacancy_salary_currency = 'Nan'
                vacancy_max_salary = 'Nan'
                vacancy_max_salary = 'Nan'

            vacancy_min_salary = vacancy_min_salary.replace('от ', '')
            vacancy_min_salary = vacancy_min_salary.replace('руб', '')
            vacancy_min_salary = vacancy_min_salary.replace('US', '')
            vacancy_min_salary = vacancy_min_salary.replace('EU', '')
            vacancy_min_salary = vacancy_min_salary.replace('EUR', '')
            vacancy_min_salary = vacancy_min_salary.replace(' ', '')

            vacancy_max_salary = vacancy_max_salary.replace('до ', '')
            vacancy_max_salary = vacancy_max_salary.replace('руб.', '')
            vacancy_max_salary = vacancy_max_salary.replace('US', '')
            vacancy_max_salary = vacancy_max_salary.replace('EU', '')
            vacancy_max_salary = vacancy_max_salary.replace('EUR', '')
            vacancy_max_salary = vacancy_max_salary.replace(' ', r'')

            hh_vacancy_data['name'] = hh_vacancy_name
            hh_vacancy_data['link'] = hh_vacancy_link
            hh_vacancy_data['salary'] = hh_vacancy_salary
            hh_vacancy_data['min_salary'] = vacancy_min_salary
            hh_vacancy_data['max_salary'] = vacancy_max_salary
            hh_vacancy_data['vacancy_salary_currency'] = vacancy_salary_currency
            hh_vacancy_data['vacancy_site'] = 'hh.ru'
            vacancies.append(hh_vacancy_data)

    else:
        pass
    
    
elif page == 0:
    hh_page = page
    
    while hh_page < 40:
        
        hh_response = requests.get(f'{main_link_hh}{vacancy_title}&page={hh_page}', headers = header)
        hh_page += 1
        
        if hh_response.ok:
            hh_html = hh_response.text
            hh_soup = bs(hh_html, 'lxml')
            hh_vacancy_block = hh_soup.find_all('div',{'class':"vacancy-serp"})[0]
            hh_vacancies_list = hh_vacancy_block.find_all('div',{'class':"vacancy-serp-item"})

            for vacancy in hh_vacancies_list:
                vacancy_data={}
                vacancy_link = vacancy.find('a')['href']
                vacancy_name = vacancy.find('span',{'class': 'resume-search-item__name'}).getText()
                vacancy_salary = vacancy.find('div',{'class': 'vacancy-serp-item__sidebar'}).getText()
                vacancy_salary = unicodedata.normalize("NFKD", vacancy_salary)

                if 'от ' in vacancy_salary or '-' in vacancy_salary:
                    vacancy_min_salary = vacancy_salary
                    vacancy_min_salary = vacancy_min_salary[:vacancy_min_salary.find('-')]
                else:
                    vacancy_min_salary = 'Nan'


                if 'до ' in vacancy_salary or '-' in vacancy_salary:
                    vacancy_max_salary = vacancy_salary
                    vacancy_max_salary = vacancy_max_salary[vacancy_max_salary.find('-')+1:]
                else:
                    vacancy_max_salary = 'Nan'

                if ' руб' in vacancy_salary or 'US' in vacancy_salary or 'EU' in vacancy_salary:
                    vacancy_salary_currency = vacancy_salary
                    vacancy_salary_currency = vacancy_salary_currency[-4:]
                    vacancy_salary_currency = vacancy_salary_currency.replace('.', '')
                else:
                    vacancy_salary_currency = 'Nan'    

                if 'По договорённости' in vacancy_salary:
                    vacancy_salary_currency = 'Nan'
                    vacancy_max_salary = 'Nan'
                    vacancy_max_salary = 'Nan'
                else:
                    pass
                
                vacancy_min_salary = vacancy_min_salary.replace('от ', '')
                vacancy_min_salary = vacancy_min_salary.replace('руб', '')
                vacancy_min_salary = vacancy_min_salary.replace('US', '')
                vacancy_min_salary = vacancy_min_salary.replace('EU', '')
                vacancy_min_salary = vacancy_min_salary.replace('EUR', '')
                vacancy_min_salary = vacancy_min_salary.replace(' ', '')

                vacancy_max_salary = vacancy_max_salary.replace('до ', '')
                vacancy_max_salary = vacancy_max_salary.replace('руб.', '')
                vacancy_max_salary = vacancy_max_salary.replace('US', '')
                vacancy_max_salary = vacancy_max_salary.replace('EU', '')
                vacancy_max_salary = vacancy_max_salary.replace('EUR', '')
                vacancy_max_salary = vacancy_max_salary.replace(' ', r'')

                vacancy_data['name'] = vacancy_name
                vacancy_data['link'] = vacancy_link
                vacancy_data['salary'] = vacancy_salary
                vacancy_data['min_salary'] = vacancy_min_salary
                vacancy_data['max_salary'] = vacancy_max_salary
                vacancy_data['vacancy_salary_currency'] = vacancy_salary_currency
                vacancy_data['vacancy_site'] = 'hh.ru'
                vacancies.append(vacancy_data)
        
        continue
else:
    print('Вы ввели не правильное число')


In [8]:
page

0

In [9]:
print(len(vacancies))

2000



# Superjob

def transliteration(text):
    cyrillic = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
    latin = 'a|b|v|g|d|e|e|zh|z|i|i|k|l|m|n|o|p|r|s|t|u|f|kh|tc|ch|sh|shch||y||e|iu|ia'.split('|')
    trantab = {k:v for k,v in zip(cyrillic,latin)}
    newtext = ''
    for ch in text:
        casefunc =  str.capitalize if ch.isupper() else str.lower
        newtext += casefunc(trantab.get(ch.lower(),ch))
    return newtext

vacancy_title = transliteration(vacancy_title)

In [10]:
main_link_sj = ('https://www.superjob.ru/vacancy/search/?keywords=')

In [11]:
if 1 <= page < 50:
    sj_page = page
    sj_response = requests.get(f'{main_link_sj}{vacancy_title}&geo%5Bt%5D%5B0%5D=4&page={sj_page}', headers = header)
    sj_html = sj_response.text
    sj_soup = bs(sj_html, 'lxml')
    sj_vacancy_block = sj_soup.find_all('div',{'class':"_1ID8B"})[0]
    sj_vacancies_list = sj_vacancy_block.find_all('div',{'class':"QiY08 LvoDO"})
    
    if sj_response.ok:
        
        for vacancy in sj_vacancies_list:
            sj_vacancy_data={}
            sj_vacancy_link = 'https://www.superjob.ru/'+vacancy.find('a')['href']
            sj_vacancy_name = vacancy.find('a').getText()
            sj_vacancy_salary = vacancy.find('span',{'class':'_3mfro _2Wp8I _31tpt f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'}).getText()
            sj_vacancy_salary = unicodedata.normalize("NFKD", sj_vacancy_salary)

            
            if 'По договорённости' in sj_vacancy_salary:
                vacancy_min_salary = 'Nan'
                vacancy_max_salary = 'Nan'
                vacancy_salary_currency = 'Nan'
                
            if 'от' in sj_vacancy_salary or '-' in sj_vacancy_salary:
                vacancy_min_salary = sj_vacancy_salary
                vacancy_min_salary = vacancy_min_salary[:vacancy_min_salary.find('-')]
            else:
                vacancy_min_salary = 'Nan'


            if 'до ' in sj_vacancy_salary or '-' in sj_vacancy_salary:
                vacancy_max_salary = sj_vacancy_salary
                vacancy_max_salary = vacancy_max_salary[vacancy_max_salary.find('-')+1:]
            
            else:
                vacancy_max_salary = 'Nan'
            
                
            if ' руб' in sj_vacancy_salary or 'US' in sj_vacancy_salary or 'EU' in sj_vacancy_salary:
                vacancy_salary_currency = sj_vacancy_salary
                vacancy_salary_currency = vacancy_salary_currency[-4:]
                vacancy_salary_currency = vacancy_salary_currency.replace('.', '')
            else:
                vacancy_salary_currency = 'Nan'    
                     

            vacancy_min_salary = vacancy_min_salary.replace('от ', '')
            vacancy_min_salary = vacancy_min_salary.replace(' руб', '')
            vacancy_min_salary = vacancy_min_salary.replace(' US', '')
            vacancy_min_salary = vacancy_min_salary.replace(' EU', '')
            vacancy_min_salary = vacancy_min_salary.replace(' EUR', '')
            vacancy_min_salary = vacancy_min_salary.replace(' ', '')

            vacancy_max_salary = vacancy_max_salary.replace('до ', '')
            vacancy_max_salary = vacancy_max_salary.replace(' руб.', '')
            vacancy_max_salary = vacancy_max_salary.replace(' US', '')
            vacancy_max_salary = vacancy_max_salary.replace(' EU', '')
            vacancy_max_salary = vacancy_max_salary.replace(' EUR', '')
            vacancy_max_salary = vacancy_max_salary.replace(' ', r'')

            sj_vacancy_data['name'] = sj_vacancy_name
            sj_vacancy_data['link'] = sj_vacancy_link
            sj_vacancy_data['salary'] = sj_vacancy_salary
            sj_vacancy_data['min_salary'] = vacancy_min_salary
            sj_vacancy_data['max_salary'] = vacancy_max_salary
            sj_vacancy_data['vacancy_salary_currency'] = vacancy_salary_currency
            sj_vacancy_data['vacancy_site'] = 'superjob.ru'
            vacancies.append(sj_vacancy_data)
    else:
        pass
    
    
elif page == 0:
    sj_page = page+1
    
    while sj_page <= 40:
        
        sj_response = requests.get(f'{main_link_sj}{vacancy_title}&geo%5Bt%5D%5B0%5D=4&page={sj_page}', headers = header)
    
        
        try:
            sj_html = sj_response.text
            sj_soup = bs(sj_html, 'lxml')
            sj_vacancy_block = sj_soup.find_all('div',{'class':"_1ID8B"})[0]
            sj_vacancies_list = sj_vacancy_block.find_all('div',{'class':"QiY08 LvoDO"})
            
            sj_page += 1
            
            for vacancy in sj_vacancies_list:
                sj_vacancy_data={}
                sj_vacancy_link = 'https://www.superjob.ru/'+vacancy.find('a')['href']
                sj_vacancy_name = vacancy.find('a').getText()
                sj_vacancy_salary = vacancy.find('span',{'class':'_3mfro _2Wp8I _31tpt f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'}).getText()
                sj_vacancy_salary = unicodedata.normalize("NFKD", sj_vacancy_salary)

                if 'По договорённости' in sj_vacancy_salary:
                    vacancy_min_salary = 'Nan'
                    vacancy_max_salary = 'Nan'
                    vacancy_salary_currency = 'Nan'
                
                if 'от' in sj_vacancy_salary or '-' in sj_vacancy_salary:
                    vacancy_min_salary = sj_vacancy_salary
                    vacancy_min_salary = vacancy_min_salary[:vacancy_min_salary.find('-')]
                else:
                    vacancy_min_salary = 'Nan'

                if 'до ' in sj_vacancy_salary or '-' in sj_vacancy_salary:
                    vacancy_max_salary = sj_vacancy_salary
                    vacancy_max_salary = vacancy_max_salary[vacancy_max_salary.find('-')+1:]

                else:
                    vacancy_max_salary = 'Nan'

                if ' руб' in sj_vacancy_salary or 'US' in sj_vacancy_salary or 'EU' in sj_vacancy_salary:
                    vacancy_salary_currency = sj_vacancy_salary
                    vacancy_salary_currency = vacancy_salary_currency[-4:]
                    vacancy_salary_currency = vacancy_salary_currency.replace('.', '')
                else:
                    vacancy_salary_currency = 'Nan'    



                vacancy_min_salary = vacancy_min_salary.replace('от ', '')
                vacancy_min_salary = vacancy_min_salary.replace(' руб', '')
                vacancy_min_salary = vacancy_min_salary.replace(' US', '')
                vacancy_min_salary = vacancy_min_salary.replace(' EU', '')
                vacancy_min_salary = vacancy_min_salary.replace(' EUR', '')
                vacancy_min_salary = vacancy_min_salary.replace(' ', '')

                vacancy_max_salary = vacancy_max_salary.replace('до ', '')
                vacancy_max_salary = vacancy_max_salary.replace(' руб.', '')
                vacancy_max_salary = vacancy_max_salary.replace(' US', '')
                vacancy_max_salary = vacancy_max_salary.replace(' EU', '')
                vacancy_max_salary = vacancy_max_salary.replace(' EUR', '')
                vacancy_max_salary = vacancy_max_salary.replace(' ', r'')

                sj_vacancy_data['name'] = sj_vacancy_name
                sj_vacancy_data['link'] = sj_vacancy_link
                sj_vacancy_data['salary'] = sj_vacancy_salary
                sj_vacancy_data['min_salary'] = vacancy_min_salary
                sj_vacancy_data['max_salary'] = vacancy_max_salary
                sj_vacancy_data['vacancy_salary_currency'] = vacancy_salary_currency
                sj_vacancy_data['vacancy_site'] = 'superjob.ru'
                vacancies.append(sj_vacancy_data)
        
        except IndexError:
            break
else:
    print('Вы ввели не правильное число')

In [12]:
print(len(vacancies))

2051


In [14]:
df = pd.DataFrame(vacancies)

In [18]:
df

,name,link,salary,min_salary,max_salary,vacancy_salary_currency,vacancy_site
0,Бизнес-аналитик (it сервисы),https://hh.ru/vacancy/36388405?query=%D0%B0%D0...,,Nan,Nan,Nan,hh.ru
1,Маркетолог по продвижению онлайн-продукта на р...,https://hh.ru/vacancy/36666532?query=%D0%B0%D0...,от 2 000 USD,2000,Nan,USD,hh.ru
2,Системный аналитик,https://hh.ru/vacancy/36624566?query=%D0%B0%D0...,120 000-180 000 руб.,120000,180000,руб,hh.ru
3,Менеджер по контекстной рекламе,https://hh.ru/vacancy/35698510?query=%D0%B0%D0...,от 40 000 руб.,40000,Nan,руб,hh.ru
4,SEO-специалист,https://hh.ru/vacancy/36556852?query=%D0%B0%D0...,от 100 000 руб.,100000,Nan,руб,hh.ru
...,...,...,...,...,...,...,...
2046,Медицинский аналитик / Специалист в медицинско...,https://www.superjob.ru//vakansii/medicinskij-...,от 60 000 руб.,60000,Nan,руб,superjob.ru
2047,Ведущий экономист-аналитик / Главный экономист,https://www.superjob.ru//vakansii/veduschij-ek...,По договорённости,Nan,Nan,Nan,superjob.ru
2048,"Аналитик, специалист аналитико-снабженческого ...",https://www.superjob.ru//vakansii/analitik-271...,от 75 000 руб.,75000,Nan,руб,superjob.ru
2049,Математик-аналитик,https://www.superjob.ru//vakansii/matematik-an...,По договорённости,Nan,Nan,Nan,superjob.ru
